# Import Libraries


In [955]:
import random
import time
import csv

import numpy as np
import pandas as pd


# Define Classes

In [956]:
class Customer:
    def __init__(self, id, lat, lon, demand):
        self.id = id
        self.lat = lat
        self.lon = lon
        self.demand = demand
        
    def distance(self, customer):
        latDis = abs(self.lat - customer.lat)
        longDis = abs(self.lon - customer.lon)
        distance = 100 * np.sqrt((latDis ** 2) + (longDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.lat) + "," + str(self.lon) + ")"

In [957]:
class Vehicle:
    def __init__(self, type):
        self.type = type
        self.routes = []
        self.rate = None
        self.capacity = None

    def addRoute(self, route):
        self.routes.append(route)

    def setRateCapacity(self):
        if self.type == 'A':
            self.rate = 1.2
            self.capacity = 25
        elif self.type == 'B':
            self.rate = 1.5
            self.capacity = 30
        else:
            self.rate = 0
            self.capacity = 0
    
    def setStartPoint(self):
        self.routes.append(Customer(-1, 4.4184, 114.0932, 0))

In [958]:
class Fitness:
    def __init__(self, vehicle):
        self.vehicle = vehicle
        self.fitness = None
        self.distance = None
    
    # fitness is based on cost, since distance is directly in correlation with cost,
    # taking cost can be a more accurate representation of the efficiency of the route

    # routes = customer
    
    
    def getDistance(self):
        routes = self.vehicle.routes
        distance = 0

        for i in range (0, len(routes)):
            fromCity = routes[i]
            if i+1 < len(routes):
                toCity = routes[i+1]

            # return to depot
            else:
                toCity = routes[0]
            
            distance += fromCity.distance(toCity)

        return distance

    def getCost(self):
        self.distance = self.getDistance()
        return self.distance*(self.vehicle.rate)  
    
    def getDemand(self):
        routes = self.vehicle.routes
        totalDemand = 0

        for route in routes:
            totalDemand += route.demand

        return totalDemand
            

In [959]:
# fill in routes - choose vehicle - ensure routes is selected based on demands


# Generate Population

In [960]:
data = pd.read_csv('data.csv')
data.head()

li = data.values.tolist()

oriRoutes = []
for i in li:
    oriRoutes.append(Customer(i[0], i[1], i[2], i[3]))

solutions = [] #array of vehicles

for s in range(1000):
    print('iteration'+ str(s+1))
    routes = oriRoutes.copy()

    vehicles = []

    while(len(routes) !=0):
        # Randomly choose a vehicle
        type = random.choice(['A', 'B'])
        vehicle = Vehicle(type)
        vehicle.setStartPoint()
        vehicle.setRateCapacity()

        print('Chosen vehicle '+ vehicle.type)

        # Randomly fill in the routes
        filledCapacity = 0

        while(routes):
            selectedRoute = random.choice(routes)
            if((filledCapacity+selectedRoute.demand) <= vehicle.capacity):
                filledCapacity += selectedRoute.demand
                vehicle.addRoute(selectedRoute)
                routes.remove(selectedRoute)
                print('Route ' + str(selectedRoute.id) + ', demand ' + str(selectedRoute.demand))
            else:
                break
            

        vehicles.append(vehicle)

    solutions.append(vehicles)
    
solutions


'iteration1'
'Chosen vehicle B'
'Route 7.0, demand 3.0'
'Route 2.0, demand 8.0'
'Route 3.0, demand 3.0'
'Route 8.0, demand 6.0'
'Route 10.0, demand 8.0'
'Chosen vehicle A'
'Route 5.0, demand 5.0'
'Route 4.0, demand 6.0'
'Route 9.0, demand 5.0'
'Route 6.0, demand 8.0'
'Chosen vehicle B'
'Route 1.0, demand 5.0'
'iteration2'
'Chosen vehicle B'
'Route 9.0, demand 5.0'
'Route 10.0, demand 8.0'
'Route 6.0, demand 8.0'
'Route 8.0, demand 6.0'
'Route 7.0, demand 3.0'
'Chosen vehicle B'
'Route 3.0, demand 3.0'
'Route 4.0, demand 6.0'
'Route 5.0, demand 5.0'
'Route 1.0, demand 5.0'
'Route 2.0, demand 8.0'
'iteration3'
'Chosen vehicle A'
'Route 7.0, demand 3.0'
'Route 6.0, demand 8.0'
'Route 10.0, demand 8.0'
'Route 9.0, demand 5.0'
'Chosen vehicle A'
'Route 3.0, demand 3.0'
'Route 1.0, demand 5.0'
'Route 5.0, demand 5.0'
'Route 8.0, demand 6.0'
'Chosen vehicle A'
'Route 4.0, demand 6.0'
'Route 2.0, demand 8.0'
'iteration4'
'Chosen vehicle B'
'Route 5.0, demand 5.0'
'Route 1.0, demand 5.0'
'Route

[[<__main__.Vehicle at 0x22cd3af5970>,
 [<__main__.Vehicle at 0x22cd4761040>, <__main__.Vehicle at 0x22cd4760560>],
 [<__main__.Vehicle at 0x22cd4657740>, <__main__.Vehicle at 0x22cd46579e0>],
 [<__main__.Vehicle at 0x22cd4657b60>, <__main__.Vehicle at 0x22cd4657b30>],
 [<__main__.Vehicle at 0x22cd4655430>, <__main__.Vehicle at 0x22cd46554f0>],
 [<__main__.Vehicle at 0x22cd476d6a0>, <__main__.Vehicle at 0x22cd476d760>],
 [<__main__.Vehicle at 0x22cd476d820>, <__main__.Vehicle at 0x22cd476d8e0>],
 [<__main__.Vehicle at 0x22cd476ede0>, <__main__.Vehicle at 0x22cd476eea0>],
 [<__main__.Vehicle at 0x22cd476f3e0>, <__main__.Vehicle at 0x22cd476f4a0>],
 [<__main__.Vehicle at 0x22cd47807a0>, <__main__.Vehicle at 0x22cd4780860>],
 [<__main__.Vehicle at 0x22cd4780920>, <__main__.Vehicle at 0x22cd47809e0>],
 [<__main__.Vehicle at 0x22cd4781a60>, <__main__.Vehicle at 0x22cd4781b20>],
 [<__main__.Vehicle at 0x22cd4783920>, <__main__.Vehicle at 0x22cd47839e0>],
 [<__main__.Vehicle at 0x22cd4783aa0>

In [961]:
# routes do not need to include depot at end, but shoudl include depot at start

# Tournament Selection

In [962]:
def fitness(solution):
    solutionCost = 0
    for vehicle in solution:
        fitness = Fitness(vehicle)
        cost = fitness.getCost()
        solutionCost += cost
    return solutionCost

def distance(solution):
    solutionDistance = 0
    for vehicle in solution:
        fitness = Fitness(vehicle)
        distance = fitness.getDistance()
        solutionDistance += distance
    return solutionDistance

# Randomly choose 20 samples, get one parent
def getParents(population):
    # choices = random.sample(population, 20)
    choices = population.copy()

    chosenSolutionCost = []
    for vehicles in choices:
        solutionCost = fitness(vehicles)
        chosenSolutionCost.append(solutionCost)


    minpos = chosenSolutionCost.index(min(chosenSolutionCost))
    minA = choices[minpos]
    chosenSolutionCost.pop(minpos)
    minpos = chosenSolutionCost.index(min(chosenSolutionCost))
    minB = choices[minpos]

    return minA, minB

A, B = getParents(solutions)
A[0].routes



[(4.4184,114.0932), (4.4932,114.1322), (4.4935,114.1828), (4.3818,114.2034)]

# Crossover

In [963]:
def findMissing(li):
    return sorted(set(range(1, 11)).difference(li))


def convertToOneDList(li):
    convertedList = []
    for i in range(len(li)):
        for j in range(len(li[i])):
            convertedList.append(int(li[i][j]))
    return convertedList

def getDuplicatedElements(li):
    duplicates = []
    for x in li:
        if x not in duplicates and li.count(x) >1:
            duplicates.append(x)
        
    return duplicates


In [964]:

def getCrossoverOffspring(firstParent, secondParent):
    childVehicles = []

    first = [[],[],[]]
    firstVehicles = []

    for i in range(len(firstParent)):
        #some solutions only have 2 vehicles
        if (firstParent[i].routes):
            firstParent[i].routes.pop(0)
        firstVehicles.append(firstParent[i].type)
        # for every route
        for j in range(len(firstParent[i].routes)):
            first[i].append(firstParent[i].routes[j].id)



    second = [[],[],[]]
    secondVehicles = []


    for i in range(len(secondParent)):
        #some solutions only have 2 vehicles
        if (secondParent[i].routes):
            secondParent[i].routes.pop(0)
        secondVehicles.append(secondParent[i].type)
        # for every route
        for j in range(len(secondParent[i].routes)):
            second[i].append(secondParent[i].routes[j].id)


    # Random Choice of 1,2,3,4
    choice = random.randint(1, 4)
    # Case 1 - first parent is  main, two vehicles remain, one vehicle changes
    if (choice == 1 or choice == 2):
        if (choice == 1):
            parent = first.copy()
            secondary = second.copy()
            selectedVehicleIdx = random.randint(0, len(parent)-1)
            selectedToSwapVehicle = random.choice(secondary)
            parent[selectedVehicleIdx] = selectedToSwapVehicle
            parent = convertToOneDList(parent)
            childVehicles = firstVehicles.copy()

        # Case 2 - second parent is main, two vehicles remain, one vehicle changes
        elif (choice == 2):
            parent = second.copy()
            secondary = first.copy()
            selectedVehicleIdx = random.randint(0, len(parent)-1)
            selectedToSwapVehicle = random.choice(secondary)
            parent[selectedVehicleIdx] = selectedToSwapVehicle
            parent = convertToOneDList(parent)
            childVehicles = secondVehicles.copy()

        missing = findMissing(parent)
        duplicates = getDuplicatedElements(parent)

        child = []
        for i in parent:
            if i not in child:
                child.append(i)
            else:
                if(missing):
                    child.append(missing[0])
                    missing.pop(0)

        if (missing):
            child += missing


    elif (choice == 3 or choice == 4):
        # Case 3 - take first parent as a whole, no crossover, direcly sent to mutation
        if (choice == 3):
            child = convertToOneDList(first)
            childVehicles = firstVehicles.copy()
        # Case 4 - take second parent as a whole, no crossover, direcly sent to mutation
        elif (choice == 4):
            child = convertToOneDList(second)
            childVehicles = secondVehicles.copy()

    return child, childVehicles


# Mutation

In [965]:
# Vehicle Mutation
def vehicleMutation(li):
    idx = random.randint(0, len(li)-1)
    if li[idx] == 'A':
        li[idx] = 'B'
    else:
        li[idx] = 'A'
    
    return li

# Routes Mutation - Swap
def routeMutation(li):
    if(len(li) >= 1):
        aIdx = random.randint(0, len(li)-1)
        bIdx = random.randint(0, len(li)-1)

        temp = li[aIdx]
        li[aIdx] = li[bIdx]
        li[bIdx] = temp

    return li

In [966]:
# print(childVehicles)
# print(vehicleMutation(childVehicles))

# print(child)
# print(routeMutation(child))

def getMutatedOffspring(first, second):

    child, childVehicles = getCrossoverOffspring(first, second)
    mutatedChild = routeMutation(child)
    mutatedVehicles = vehicleMutation(childVehicles)

    return mutatedChild, mutatedVehicles




# Convert back to class

In [967]:
# convert routes to Customer class
data = pd.read_csv('data.csv')
data.head()

li = data.values.tolist()
print(li)

routes = []
for i in li:
    routes.append(Customer(i[0], i[1], i[2], i[3]))


[[1.0, 4.3555, 113.9777, 5.0],
 [2.0, 4.3976, 114.0049, 8.0],
 [3.0, 4.3163, 114.0764, 3.0],
 [4.0, 4.3184, 113.9932, 6.0],
 [5.0, 4.4024, 113.9896, 5.0],
 [6.0, 4.4142, 114.0127, 8.0],
 [7.0, 4.4804, 114.0734, 3.0],
 [8.0, 4.3818, 114.2034, 6.0],
 [9.0, 4.4935, 114.1828, 5.0],
 [10.0, 4.4932, 114.1322, 8.0]]


In [968]:
def getOffspring(first, second):
    mutatedChild, mutatedVehicles = getMutatedOffspring(first, second)
    while(len(mutatedChild) != 10):
        mutatedChild, mutatedVehicles = getMutatedOffspring(first, second)
    # print(mutatedChild)
    # print(mutatedVehicles)

    offspring = []
    for i in mutatedVehicles:
        vehicle = Vehicle(i)
        vehicle.setStartPoint()
        vehicle.setRateCapacity()

        filledCapacity = 0

        while(mutatedChild):
            idx = mutatedChild[0]
            selectedRoute = routes[idx-1]
            if((filledCapacity+selectedRoute.demand) <= vehicle.capacity):
                filledCapacity += selectedRoute.demand
                vehicle.addRoute(selectedRoute)
                mutatedChild.pop(0)
            else:
                break
        
        offspring.append(vehicle)

    return offspring
    

In [969]:
import copy
# Generate two offspring
parent1, parent2 = getParents(solutions)


noIteration = 250
minScore = 1000

for i in range(noIteration):
    population = []
    # each parent will produce 1000 offspring, and two best are selected
    for j in range(500):
        offspringA = getOffspring(parent1, parent2)
        offspringB = getOffspring(parent1, parent2)

        population.append(offspringA)
        population.append(offspringB)
    

    parent1, parent2 = getParents(population)

    averageScore = fitness(parent1)

    print('Iteration: ' + str(i+1))
    print('Score: ' + str(averageScore))

    if(averageScore < minScore):
        minScore = averageScore
        minParent = copy.deepcopy(parent1)



'Iteration: 1'
'Score: 143.42708299858427'
'Iteration: 2'
'Score: 150.98313686644053'
'Iteration: 3'
'Score: 143.42708299858427'
'Iteration: 4'
'Score: 150.98313686644053'
'Iteration: 5'
'Score: 143.42708299858427'
'Iteration: 6'
'Score: 150.98313686644053'
'Iteration: 7'
'Score: 143.42708299858427'
'Iteration: 8'
'Score: 150.98313686644053'
'Iteration: 9'
'Score: 143.42708299858427'
'Iteration: 10'
'Score: 150.98313686644053'
'Iteration: 11'
'Score: 143.42708299858427'
'Iteration: 12'
'Score: 150.98313686644053'
'Iteration: 13'
'Score: 143.42708299858427'
'Iteration: 14'
'Score: 150.98313686644053'
'Iteration: 15'
'Score: 143.42708299858427'
'Iteration: 16'
'Score: 150.98313686644053'
'Iteration: 17'
'Score: 143.42708299858427'
'Iteration: 18'
'Score: 150.98313686644053'
'Iteration: 19'
'Score: 143.42708299858427'
'Iteration: 20'
'Score: 150.98313686644053'
'Iteration: 21'
'Score: 143.42708299858427'
'Iteration: 22'
'Score: 150.98313686644053'
'Iteration: 23'
'Score: 143.4270829985842

In [970]:
print("Total Distance: " + str(round(distance(minParent), 2)) + " km")
print("Total Cost: RM " + str(round(minScore, 2)))
print('')


for idx, vehicle in enumerate(minParent):
    if(vehicle):

        print(f"Vehicle {idx+1} (Type {vehicle.type})")
        vehicleFitness = Fitness(vehicle)
        print(f"Round Trip Distance: {round(vehicleFitness.getDistance(),3)} km, Cost: RM {round(vehicleFitness.getCost(),2)}, Demand: {vehicleFitness.getDemand()}")

        for idx, route in enumerate(vehicle.routes):
            
            if (route.id == -1):
                routeName = "Depot"
                print(f"{routeName} -> ")
            else:
                routeName = "C"+str(int(route.id))
                print(f"{routeName} ({round(route.distance(vehicle.routes[idx-1]),2)} km) -> ")

        backToDepotDistance = vehicle.routes[-1].distance(vehicle.routes[0])
        print(f"Depot ({round(backToDepotDistance, 2)} km)")
        print('')

'Total Distance: 107.95 km'
'Total Cost: RM 129.54'
''
'Vehicle 1 (Type A)'
'Round Trip Distance: 36.775 km, Cost: RM 44.13, Demand: 22.0'
'Depot -> '
'C3 (10.35 km) -> '
'C4 (8.32 km) -> '
'C1 (4.02 km) -> '
'C2 (5.01 km) -> '
'Depot (9.07 km)'
''
'Vehicle 2 (Type A)'
'Round Trip Distance: 36.512 km, Cost: RM 43.81, Demand: 24.0'
'Depot -> '
'C5 (10.48 km) -> '
'C6 (2.59 km) -> '
'C7 (8.98 km) -> '
'C10 (6.02 km) -> '
'Depot (8.44 km)'
''
'Vehicle 3 (Type A)'
'Round Trip Distance: 34.661 km, Cost: RM 41.59, Demand: 11.0'
'Depot -> '
'C9 (11.69 km) -> '
'C8 (11.36 km) -> '
'Depot (11.61 km)'
''
